# Loop 3 Analysis - Fix Overlap in Group 004

Submission failed with 'Overlapping trees in group 004'. Need to investigate and fix.

In [ ]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 50  # Higher precision
scale_factor = Decimal('1e15')

print('Libraries loaded')

In [ ]:
# Load the failed submission
df = pd.read_csv('/home/submission/submission.csv')
print(f'Total rows: {len(df)}')

# Extract group 004
group_004 = df[df['id'].str.startswith('004_')]
print(f'\nGroup 004 rows: {len(group_004)}')
print(group_004)

In [ ]:
# Parse coordinates
def parse_coord(s):
    return float(str(s).replace('s', ''))

group_004_parsed = []
for _, row in group_004.iterrows():
    x = parse_coord(row['x'])
    y = parse_coord(row['y'])
    deg = parse_coord(row['deg'])
    group_004_parsed.append((x, y, deg))
    print(f"{row['id']}: x={x:.15f}, y={y:.15f}, deg={deg:.15f}")

print(f'\nParsed {len(group_004_parsed)} trees')

In [ ]:
# Create tree polygon with high precision
class ChristmasTree:
    def __init__(self, center_x, center_y, angle):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def check_overlap_detailed(trees):
    """Check for overlaps and return details"""
    polygons = [t.polygon for t in trees]
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if not polygons[i].touches(polygons[j]):
                    overlaps.append((i, j, intersection.area))
    return overlaps

print('Functions defined')

In [ ]:
# Check group 004 for overlaps
trees_004 = [ChristmasTree(x, y, deg) for x, y, deg in group_004_parsed]
overlaps = check_overlap_detailed(trees_004)

if overlaps:
    print(f'OVERLAPS FOUND in group 004:')
    for i, j, area in overlaps:
        print(f'  Trees {i} and {j}: intersection area = {area}')
else:
    print('No overlaps detected in group 004 with our validation')

In [ ]:
# Let's check ALL groups for overlaps
print('Checking all groups for overlaps...')

df['N'] = df['id'].str.split('_').str[0].astype(int)

overlapping_groups = []
for n, group in df.groupby('N'):
    trees = []
    for _, row in group.iterrows():
        x = parse_coord(row['x'])
        y = parse_coord(row['y'])
        deg = parse_coord(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    
    overlaps = check_overlap_detailed(trees)
    if overlaps:
        overlapping_groups.append((n, overlaps))
        if n <= 10:  # Print details for small groups
            print(f'Group {n}: {len(overlaps)} overlaps')
            for i, j, area in overlaps:
                print(f'  Trees {i} and {j}: area = {area}')

print(f'\nTotal groups with overlaps: {len(overlapping_groups)}')
if overlapping_groups:
    print(f'Groups: {[g[0] for g in overlapping_groups][:20]}...')